In [ ]:
# Combine the Practice and Condition Phase Mouse Usage during a task together into a single merged file to be able
# to control for the "baseline" in the image data analysis

# This exists because I did not find a good solution (without a "big" rework of the existing code) that creates
# the merged images in the preparation_of_mouse_data.ipynb file directly from the separate matplotlib figures that
# are created in the create_mouse_analysis_dataset function loop

#### package import

In [ ]:
import numpy as np
import os
from PIL import Image

#### choose the task whose images need to be merged (only 1 task at the time): then run the cells one after another

In [ ]:
# chose the task
task_name = "Point_Click_Task"
# task_name = "Drag_Drop_Task"
# task_name = "Follow_Circle_Task"
# task_name = "Slider_Task"

In [ ]:
# get the path

# get the current path
dir_path = os.getcwd()

# get the directory of the folders with the images of the condition and practice phase
# this assumes the images are saved in a folder structure as following: .../Mouse_Usage_Images --> Cond or Prac --> 
# Taskname, e.g. .../Mouse_Usage_Images/Cond/Point_and_click_task
cond_directory = os.path.join(dir_path, "Mouse_Usage_Images", "Cond", task_name)
prac_directroy = os.path.join(dir_path, "Mouse_Usage_Images", "Prac", task_name)

In [ ]:
# get the image filenames in the directory
condition_images = [i for i in os.listdir(cond_directory) if ".png" in i]
practice_images = [i for i in os.listdir(prac_directroy) if ".png" in i]

In [ ]:
# get the participant ids of each file name and a list that contains the participant ids that are not in both lists
cond_par_ids = [i[-40:-13] for i in condition_images]
prac_par_ids = [i[-40:-13] for i in practice_images]

ids_not_in_both = list(set(cond_par_ids) ^ set(prac_par_ids))

In [ ]:
# remove the participants that have only an image in the practice or condition phase and not in both
def remove_single_images(image_list):

    images_to_remove = []

    # loop over the images and check if the participant id is not in the practice and condition phase
    for image in image_list:
        for id in ids_not_in_both:
            if id in image:
                images_to_remove.append(image)

    # remove the "bad" participant ids from the file list
    cleaned_list = [i for i in image_list if i not in images_to_remove]

    # sort the file list
    cleaned_list.sort()

    return cleaned_list

In [ ]:
# get the cleaned and sorted list with the images that need to be merged
cleaned_condition_images = remove_single_images(condition_images)
cleaned_practice_images = remove_single_images(practice_images)

In [ ]:
# loop over all image files and merge them into a single file
for i in range(0, len(cleaned_condition_images)):

    # get the image files
    images = [Image.open(os.path.join(cond_directory, cleaned_condition_images[i])),
              Image.open(os.path.join(prac_directroy, cleaned_practice_images[i]))]

    # pick the image which is the smallest, and resize the others to match it (can be arbitrary image shape here)
    min_shape = sorted([(np.sum(i.size), i.size) for i in images])[0][1]

    # stack the two images vertically and create the merged image
    imgs_comb = np.vstack((np.asarray(i.resize(min_shape)) for i in images))
    imgs_comb = Image.fromarray(imgs_comb)

    # resize the image to a specific width that works well with the image classification
    basewidth = 224
    wpercent = (basewidth / float(imgs_comb.size[0]))
    hsize = int((float(imgs_comb.size[1]) * float(wpercent)))
    imgs_comb = imgs_comb.resize((basewidth, hsize), Image.ANTIALIAS)

    # information to name the merged image (infos are saved in the file names and selected by the corresponding index):
    # filename structure: taskname_participant_id_condition_arousal_valence.png
    par_id = cleaned_condition_images[i][-40:-13]
    condition = cleaned_condition_images[i][-11]

    # information about valence
    pr_valence = int(cleaned_practice_images[i][-6:-4])
    con_valence = int(cleaned_condition_images[i][-6:-4])

    # save the valence value as a string (it must be considered if it is a positve or negative value in order to be
    # able to read the value from the string
    valence_diff = con_valence - pr_valence
    if valence_diff >= 0:
        valence_diff = "+" + str(valence_diff)
    else:
        valence_diff = str(valence_diff)

    # same procedure for the arousal value
    pr_arousal = int(cleaned_practice_images[i][-9:-7])
    con_arousal = int(cleaned_condition_images[i][-9:-7])

    arousal_diff = con_arousal - pr_arousal
    if arousal_diff >= 0:
        arousal_diff = "+" + str(arousal_diff)
    else:
        arousal_diff = str(arousal_diff)

    path = os.path.join(dir_path, "Mouse_Usage_Images", "Merged")

    # print the participant that is being processed and check if the participant ids of the merged images are matching
    print("Processing: " + par_id, cleaned_practice_images[i][-40:-13])

    # save the merged image file with the following name structure:
    # merged_participantid_condition_arousalDifference(+value or -value)_valenceDifference(+value or -value)
    imgs_comb.save(path + "/" + task_name + "/Merged_" + par_id + "_" + condition + "_"
                   + arousal_diff + "_" + valence_diff + ".png")